<a href="https://colab.research.google.com/github/andrewwatkinson/Projects/blob/master/ColabProjects/COVID19_GraphGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>COVID19 Graph Generator</h1>

Press the play Button on the left of the first box below, once thats done you can change the settings on the second box and it will generate the corrasponding graph

In [0]:
#@title Run this first! Only needs to be run once.
 
!git clone https://github.com/CSSEGISandData/COVID-19

In [0]:
#@title #Configure Graph { run: "auto", vertical-output: true, form-width: "500px", display-mode: "form" }
#@markdown ---
import csv
from datetime import datetime
import re
from matplotlib import pyplot
from matplotlib import dates as d
import os
from tabulate import tabulate
import numpy as np
from math import log, exp
from datetime import timedelta
import locale
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
locale.setlocale(locale.LC_ALL, 'en_US.utf8')
from mpl_toolkits.mplot3d import Axes3D
def get_data():
    path = "COVID-19/csse_covid_19_data/csse_covid_19_daily_reports"
    file_count = 0
    dates = []
    Confirmed = []
    deathList = []
    Recover = []

    for filename in os.listdir(path):
        if filename.endswith(".csv"):
            deaths = 0
            con = 0
            recoveries = 0
            file_count += 1
            country_list = []
            with open(os.path.join(path, filename)) as csv_file:
                try:
                    datepre1 = re.sub('\\.csv', '', filename)
                    datepre2 = re.sub('-', '/', datepre1)
                    date = datetime.strptime(datepre2, '%m/%d/%Y').date()
                except:
                    pass
                csv_reader = csv.reader(csv_file, delimiter=',')
                line_count = 0
                for row in csv_reader:
                    if line_count == 0:
                        line_count += 1
                    else:
                        
                        if row[1] in country_list:
                          pass
                        else:
                          country_list.append(row[1])
                        if (row[1] == location_input) | all:
                            try:
                                deaths += float(row[4])
                                recoveries += float(row[5])
                                con += float(row[3])
                            except:
                                line_count += 1
                                continue
                        line_count += 1
            dates.append(date)
            deathList.append(deaths)
            Confirmed.append(con)
            Recover.append(recoveries)
    print(f'Processed {file_count} days')
    pl = pyplot
    plot(dates, deathList, Confirmed, Recover, pl, file_count)


def plot(x, y, z, g, ax, days):
    ax.xlabel("date")
    ax
    ax.gca().xaxis.set_major_formatter(d.DateFormatter('%m/%d/%Y'))
    if DataToDisplay == 'Deaths':
        ax.ylabel("number of deaths")
        new_x, new_y = zip(*sorted(zip(x, y)))
        try:
          prex1, prey1, prex2, prey2 = Predict(new_y, new_x, days)
          ax.plot(prex1, prey1, 'b:', label = 'Min')
          ax.plot(prex2, prey2, 'g:', label = 'Max')
        except:
          print("\nnot enough Data to predict trend\ntry changing the buffer to a lower number\n")
        ax.plot(new_x, new_y, 'r-', label = 'Confirmed')
        if GraphType == 'Log':
            ax.yscale('log')
        else:
            pass
    elif DataToDisplay == 'Confirmed':
        ax.ylabel("number of cases")
        new_x, new_y = zip(*sorted(zip(x, z)))
        try:
          prex1, prey1, prex2, prey2 = Predict(new_y, new_x, days)
          ax.plot(prex1, prey1, 'b:', label = 'Min')
          ax.plot(prex2, prey2, 'g:', label = 'Max')
        except:
          print("\nnot enough Data to predict trend\ntry changing the buffer to a lower number\n")
        ax.plot(new_x, new_y, 'r-', label = 'Confirmed')
        if GraphType == 'Log':
            ax.yscale('log')
        else:
            pass
    elif DataToDisplay == 'Recoveries':
        ax.ylabel("number of deaths")
        new_x, new_y = zip(*sorted(zip(x, g)))
        try:
          prex1, prey1, prex2, prey2 = Predict(new_y, new_x, days)
          ax.plot(prex1, prey1, 'b:', label = 'Min')
          ax.plot(prex2, prey2, 'g:', label = 'Max')
        except:
          print("\nnot enough Data to predict trend\ntry changing the buffer to a lower number\n")
        ax.plot(new_x, new_y, 'r-', label = 'Confirmed')
        if GraphType == 'Log':
            ax.yscale('log')
        else:
            pass
    ax.grid(True)
    ax.legend()
    ax.gcf().autofmt_xdate()
    pyplot.show()


def Predict( y, x, days):
  #@markdown ##Prediction settings
  #@markdown move the slider to select the number of days to predict
  numberOfDays = 7 #@param {type: "slider", min: 1, max: 7}
  #@markdown Select a specific date to predict
  dateToPredict = "2020-03-20" #@param {type:"date"}
  #@markdown ---
  d1 = datetime.strptime(dateToPredict, "%Y-%m-%d").date()
  start = 0
  end = 0
  minV = 10000000000000000000000000
  maxV = 0

  for g in range(1,15):
    try:
      cd = days+1
      deltaY = log(y[len(y)-1]/y[len(y)-(g+1)]) 
      deltaX= log(days/(days-g))
      m = deltaY/deltaX
      c = log(y[len(y)-1])-(m*log(days))
      f = lambda a : exp(m*log(a) + c)
      current = f(cd)
      if current > maxV:
        maxV = current
        end = g
      if current < minV:
        minV = current
        start = g
    except:
      break
  deltaY_rangestart = log(y[len(y)-1]/y[len(y)-(start+1)]) 
  deltaX_rangestart = log(days/(days-start))

  deltaY_rangeend = log(y[len(y)-1]/y[len(y)-(end+1)]) 
  deltaX_rangeend = log(days/(days-end))

  m1 = deltaY_rangestart/deltaX_rangestart
  m2 = deltaY_rangeend/deltaX_rangeend

  c1 = log(y[len(y)-1])-(m1*log(days))
  c2 = log(y[len(y)-1])-(m2*log(days))

  f1 = lambda a : exp(m1*log(a) + c1)
  f2 = lambda a : exp(m2*log(a) + c2)

  holder = y[len(y)-1]
  Predict_dataX1 = []
  Predict_dataY1 = []
  Predict_dataX2 = []
  Predict_dataY2 = []
  current_x = x[len(x)-1]
  diff = (d1 - current_x).days+ days
  Predict_dataY1.append(holder)
  Predict_dataY2.append(holder)
  Predict_dataX1.append(current_x)
  Predict_dataX2.append(current_x)
  DataToTable = []
  Columns = ['Day','Min', 'Max']
  for l in range(1,numberOfDays+1):
    current_day = days+l
    pre_y1 = f1(current_day)
    pre_y2 = f2(current_day)
    Predict_dataY1.append(pre_y1)
    Predict_dataY2.append(pre_y2)
    c_x = current_x + timedelta(days=l)
    Predict_dataX1.append(c_x)
    Predict_dataX2.append(c_x)
    p1 = locale.format("%d", int(pre_y1), grouping=True)
    p2 = locale.format("%d", int(pre_y2), grouping=True)
    DataToTable.append([c_x.strftime("%d %b %Y"), p1, p2])
  print("last day processed: "+current_x.strftime("%d %b %Y"))
  print("\n\nnumber on "+d1.strftime("%d %b %Y")+" will be between: ", locale.format("%d",int(f1(diff)), grouping = True),"and", locale.format("%d",int(f2(diff)), grouping = True))
  print("\n")
  print(tabulate(DataToTable, headers = Columns, tablefmt='orgtbl'), end="")
  print("\n\n")
  return Predict_dataX1, Predict_dataY1, Predict_dataX2, Predict_dataY2

Scope = 'World' #@param ["World", "Country"]
DataToDisplay = 'Confirmed' #@param ["Deaths", "Confirmed", "Recoveries"]
GraphType = 'Linear' #@param ["Log", "Linear"]
if Scope == 'World':
    all = True
    location_input = "none"
else:
    all = False
    location_input = "Iran" #@param ["UK", "Italy", "Iran", "Mainland China", "Taiwan", "Hong Kong", "Macau", "US", "Japan", "Thailand", "South Korea", "Singapore", "Vietnam", "France", "Nepal", "Malaysia", "Canada", "Cambodia", "Sri Lanka", "Australia", "Germany"]
get_data()





